### Retrieves all regions that work with EC2

In [ ]:
# pip install boto3
import boto3, time # see https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html
print('Regions:', boto3.Session().get_available_regions('ec2')) 

us-east-1       @  N. Virginia     
us-east-2       @  Ohio            
us-west-1       @  N. California   
us-west-2       @  Oregon          
af-south-1      @  Cape Town       
ap-east-1       @  Hong Kong       
ap-south-1      @  Mumbai          
ap-northeast-2  @  Seoul           
ap-southeast-1  @  Singapore       
ap-southeast-2  @  Sydney          
ap-northeast-1  @  Tokyo           
ca-central-1    @  Central Canada         
eu-central-1    @  Frankfurt       
eu-west-1       @  Ireland         
eu-west-2       @  London          
eu-south-1      @  Milan           
eu-west-3       @  Paris           
eu-north-1      @  Stockholm       
me-south-1      @  Bahrain         
sa-east-1       @  São Paulo   

### Create a session and play at Oregon (us-west-2)

In [ ]:
import secret # my personal variables 
aws_access_key_id     = secret.aws_access_key_id
aws_secret_access_key = secret.aws_secret_access_key  

region_name = 'us-west-2'
session = boto3.Session(region_name           = region_name          , 
                        aws_access_key_id     = aws_access_key_id    , 
                        aws_secret_access_key = aws_secret_access_key)

### Create a client 

In [ ]:
client = session.client('ec2')

### Create a resource  

In [ ]:
resource = session.resource('ec2')

### Create a ssh key pair for the region of Oregon

In [ ]:
myKeyPair = resource.create_key_pair(KeyName='my_keypair_at_Oregon')

### Change firewall rule to allow inbound ssh connection

In [ ]:
VpcId = client.describe_vpcs(
    Filters=[{'Name': 'isDefault', 'Values': ['true']}] 
)['Vpcs'][0]['VpcId']

GroupId = client.describe_security_groups(
    Filters=[{'Name': 'vpc-id', 'Values': [VpcId]}, {'Name': 'group-name', 'Values': ['default']}]
)['SecurityGroups'][0]['GroupId']

sg = resource.SecurityGroup(GroupId)
_ = sg.revoke_ingress(IpPermissions=sg.ip_permissions) # delete default permission  
_ = sg.authorize_ingress(IpPermissions=[               # create new one
                         {'IpProtocol': 'tcp',
                          'FromPort': 22,
                          'ToPort': 22,
                          'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
]) 

### Get the latest image of Ubuntu 20.04

In [ ]:
ubuntu_20_04_images = client.describe_images(    
    Filters=[
        { 'Name': 'architecture', 'Values': [ 'x86_64', ] },
        { 'Name': 'image-type', 'Values': [ 'machine', ] },
        { 'Name': 'state', 'Values': [ 'available', ] }, 
        { 'Name': 'name', 'Values': [ 'ubuntu/images/hvm-ssd/ubuntu-focal-20.04-amd64-server-????????', ] },]
)['Images'] 
from operator import itemgetter 
image_id_of_latest_ubuntu_20_04 = sorted(ubuntu_20_04_images, key=itemgetter('CreationDate'), reverse=True)[0]['ImageId']

### Run the instance

In [ ]:
instanceInfo = resource.create_instances( 
    KeyName=myKeyPair.key_name,              # ssh key
    ImageId=image_id_of_latest_ubuntu_20_04,  
    InstanceType='t2.micro',
    MaxCount=1,
    MinCount=1,
    EbsOptimized=False,   
    NetworkInterfaces=[
        {    
            'AssociatePublicIpAddress': True,
            'DeleteOnTermination': True, 
            'DeviceIndex': 0,
        }, 
    ] 
)

### IP and private key for SSH connection

In [ ]:
myInstance = resource.Instance(instanceInfo[0].instance_id)
myInstance.wait_until_running()
print('Public IP:', myInstance.public_ip_address)
print(myKeyPair.key_material)

# Delete the instance !!!

In [ ]:
myInstance.terminate()